In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import datetime
import glob

In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 2009
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:24:33
2025-02-17 01:25:00
2025-02-17 01:25:23
2025-02-17 01:25:42
2025-02-17 01:26:02
2025-02-17 01:26:22
2025-02-17 01:26:43
2025-02-17 01:27:02
2025-02-17 01:27:23
2025-02-17 01:27:41
2025-02-17 01:28:05
2025-02-17 01:28:28
2025-02-17 01:28:53
2025-02-17 01:29:15
2025-02-17 01:29:34
2025-02-17 01:29:56
2025-02-17 01:30:19
2025-02-17 01:30:41
2025-02-17 01:31:02
2025-02-17 01:31:22
2025-02-17 01:31:43
2025-02-17 01:32:07
2025-02-17 01:32:26
2025-02-17 01:32:56
2025-02-17 01:33:31
2025-02-17 01:33:59
2025-02-17 01:34:20
2025-02-17 01:34:47
2025-02-17 01:35:07
2025-02-17 01:35:35
2025-02-17 01:36:02
2025-02-17 01:36:20
2025-02-17 01:36:23
2025-02-17 01:36:38
2025-02-17 01:36:58
2025-02-17 01:37:28
2025-02-17 01:37:54
2025-02-17 01:38:24
2025-02-17 01:38:53
2025-02-17 01:39:19
2025-02-17 01:39:45
2025-02-17 01:40:08
2025-02-17 01:40:32
2025-02-17 01:41:00
2025-02-17 01:41:29
2025-02-17 01:41:54
2025-02-17 01:42:21
2025-02-17 01:42:48
2025-02-17 01:43:12
2025-02-17 01:43:37


In [2]:
def create_ds(ds, array):
    lonn = ds.lon.values
    latt = ds.lat.values

    ds_ = xr.Dataset({'s': ([ 'lat', 'lon'], array)},
                    coords={'lon': (['lon'], lonn),
                            'lat': (['lat'], latt),})
    return ds_

In [3]:
import numpy as np
import scipy.stats as stats

def calculate_ccscale_slope(temperature_data, precipitation_data):
    # Ensure the data is flat (1D)
    temperature_data = np.ravel(temperature_data)
    precipitation_data = np.ravel(precipitation_data)

    # Early exit if input arrays are empty
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan, np.nan, np.nan, np.nan

    # Remove data points where either temperature or precipitation contains NaN
    valid_mask = ~np.isnan(temperature_data) & ~np.isnan(precipitation_data)
    temperature_data = temperature_data[valid_mask]
    precipitation_data = precipitation_data[valid_mask]
    
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan, np.nan, np.nan, np.nan
    
    # Convert Kelvin temperatures to Celsius
    temperature_data_celsius = temperature_data - 273.15

    # Set the temperature bin size and sliding step
    bin_size = 1.0  # Bin size
    step = 0.5      # Sliding step
    min_temp = temperature_data_celsius.min()
    max_temp = temperature_data_celsius.max()
    
    
    # Create sliding temperature bins
    temperature_bins = np.arange(min_temp, max_temp, step)
    overlapping_bins = [(start, start + bin_size) for start in temperature_bins]

    # Collect precipitation data for each temperature bin using a dictionary
    precipitation_per_bin = {i: [] for i in range(len(overlapping_bins))}
    for temp, precip in zip(temperature_data_celsius, precipitation_data):
        for i, (bin_start, bin_end) in enumerate(overlapping_bins):
            if bin_start <= temp < bin_end:
                precipitation_per_bin[i].append(precip)

    # Calculate the 99th percentile of log precipitation, mean temperature, and confidence intervals
    log_precipitation_99 = []
    mean_temperatures = []
    for idx, (bin_start, bin_end) in enumerate(overlapping_bins):
        bin_data = precipitation_per_bin[idx]
        if len(bin_data) >= 80:
            log_precip = np.log(bin_data)
            quantile_99 = np.percentile(log_precip, 99)
            mean_temp = (bin_start + bin_end) / 2
            log_precipitation_99.append(quantile_99)
            mean_temperatures.append(mean_temp)

    # Perform linear regression if we have at least 5 bins with sufficient data
    if len(log_precipitation_99) >= 5:
        slope, intercept, r_value, p_value, std_err = stats.linregress(mean_temperatures, log_precipitation_99)
    else:
        slope = np.nan
        intercept = np.nan

    return mean_temperatures, log_precipitation_99, slope, intercept

# Example usage:
# temperature_data = np.array([...])
# precipitation_data = np.array([...])
# results = calculate_ccscale_slope(temperature_data, precipitation_data)
# print(results)



In [4]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def calculate_ccscale_quantreg(temperature_data, precipitation_data):
    # 确保数据是平的（一维的）
    temperature_data = np.ravel(temperature_data)
    precipitation_data = np.ravel(precipitation_data)

    # 移除温度或降水量中包含NaN的数据点
    valid_mask = ~np.isnan(temperature_data) & ~np.isnan(precipitation_data)
    temperature_data = temperature_data[valid_mask]
    precipitation_data = precipitation_data[valid_mask]
    # Early exit if input arrays are empty
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan

    # 将开尔文温度转换为摄氏度
    temperature_data_celsius = temperature_data - 273.15

    # 创建一个 DataFrame 来包含温度和降水量的数据
    data = pd.DataFrame({
        'Temperature': temperature_data_celsius,
        'Precipitation': precipitation_data
    })

    # 定义模型：在 0.99 分位数处进行分位数回归
    quantile_model = sm.QuantReg(np.log(data['Precipitation']), sm.add_constant(data['Temperature']))
    quantile_regression_result = quantile_model.fit(q=0.99).params[1]

    return quantile_regression_result

# 使用实际的 temperature_data 和 precipitation_data 调用该函数
# 例如：
# result = calculate_ccscale_quantreg(temperature_data, precipitation_data)
# print(result.summary())


In [9]:

folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
     'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
            slope_nt = calculate_ccscale_slope(temperature_data_nt, precipitation_data_nt)[2]
            slope_dt = calculate_ccscale_slope(temperature_data_dt, precipitation_data_dt)[2]
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_lrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_lrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_lrs_nt.to_netcdf(input_folder_t+'ds_lrs_nt'+folder+'.nc')
    ds_lrs_dt.to_netcdf(input_folder_t+'ds_lrs_dt'+folder+'.nc')

05:03:00.047955
05:08:29.039651
05:16:39.062601
05:28:24.274513
05:44:54.894212
06:05:31.213883
06:27:01.741411
06:48:14.725636
07:03:47.642264
07:13:16.040123
07:19:48.869892
07:31:26.376545
07:47:12.384054
08:05:54.131621
08:29:06.232491
08:55:24.963976
09:23:39.241414
09:50:47.320906
09:59:45.070732
10:08:24.572888
10:20:51.625262
10:38:40.528051
11:00:31.036875
11:23:46.468728
11:46:56.372975
12:13:17.988387
12:40:13.522797
12:53:52.925741
13:01:11.979247
13:10:01.364885
13:17:24.317068
13:27:20.168104
13:37:57.939084
13:48:36.407261
14:00:02.322439
14:14:09.253070


In [12]:
folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
     'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
    
            slope_nt = calculate_ccscale_quantreg(temperature_data_nt, precipitation_data_nt)
            slope_dt = calculate_ccscale_quantreg(temperature_data_dt, precipitation_data_dt)
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_qrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_qrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_qrs_nt.to_netcdf(input_folder_t+'ds_qrs_nt'+folder+'.nc')
    ds_qrs_dt.to_netcdf(input_folder_t+'ds_qrs_dt'+folder+'.nc')

16:25:58.493933


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

16:31:56.010404


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

16:43:15.625272


...

/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

01:00:32.943344


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

01:16:00.567330


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

In [ ]:
input_folder_t = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'


folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_dewtemp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
            slope_nt = calculate_ccscale_slope(temperature_data_nt, precipitation_data_nt)[2]
            slope_dt = calculate_ccscale_slope(temperature_data_dt, precipitation_data_dt)[2]
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_lrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_lrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_lrs_nt.to_netcdf(input_folder_t+'ds_dewlrs_nt'+folder+'.nc')
    ds_lrs_dt.to_netcdf(input_folder_t+'ds_dewlrs_dt'+folder+'.nc')

In [3]:
ds = xr.open_dataset('/N/project/Zli_lab/gongg/CONUS404_data/LST/Midwest/mt_1986_06_23.nc')